## Modelo de datos de biblioteca de música

Este modelo de datos representa una biblioteca de música que tiene una gran colección de canciones y muestra sus canciones más descargadas casi en tiempo real.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un disco de canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de descarga específico o todos los registros de descarga de una canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de recuento mensual específico de descargas o todos los registros de recuento de descargas mensuales de una canción, facilitada por una tabla llamada Song.

* Recuperación de todos los registros (incluidos los registros de canciones, los registros de descargas y los registros de recuento de descargas mensuales) de una canción, facilitada por una tabla llamada Songs.

* Buscar la mayoría de las canciones descargadas, facilitado por el índice secundario global de la tabla Canciones llamado DownloadsByMonth.

In [8]:
import pandas as pd
from dynamodb import DynamoTable
from time import sleep
import json
from decimal import Decimal

In [3]:
dt = DynamoTable()
try:
    dt.select_table('Songs')
    print(dt)
except:
    dt.create_table(
        table_name='Songs',
        partition_key='Id',
        partition_key_type='S',
        sort_key='Metadata',
        sort_key_type='S',
        provisioned=False
    )

Table created successfully!


In [4]:
dt.load_json("songs.json")

In [5]:
df = dt.query_items(query="1", to_pandas=True, consumed_capacity=True)
df

Consumed Capacity: 0.5


,Title,Id,TotalDownloads,Metadata,Artist,DownloadTimestamp,DownloadMonth,TotalDownloadsInMonth
0,One,1,3,Details,Metallica,NaN,NaN,NaN
1,NaN,1,NaN,Dld-9349823681,NaN,2018-01-01T00:00:07,NaN,NaN
2,NaN,1,NaN,Dld-9349823682,NaN,2018-01-01T00:20:10,NaN,NaN
3,NaN,1,NaN,Dld-9349823683,NaN,2018-01-02T00:11:10,NaN,NaN
4,NaN,1,NaN,Month-01-2018,NaN,NaN,01-2018,3


* DynamoDB permite la creación de índices secundarios para obtener más patrones de acceso a los datos en la tabla. Los índices secundarios son un medio eficaz para agregar flexibilidad de consulta a la tabla de DynamoDB.

* DynamoDB tiene dos tipos de índices secundarios: los índices secundarios globales y los índices secundarios locales. En esta sección, se agrega un índice secundario global al atributo de categoría.

In [6]:
dt.create_global_secondary_index(
    att_name="DownloadMonth",
    att_type="S",
    sort_index="TotalDownloadsInMonth",
    sort_type="N",
    i_name="DownloadsByMonth",
    provisioned=False,
    i_rcu=1,
    i_wcu=1
)

In [9]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


Crear un índice secundario global es muy parecido a crear una tabla. Se especifican el nombre del índice, los atributos que estarán dentro del índice, el esquema de la clave del índice y el rendimiento aprovisionado (la capacidad máxima que una aplicación puede consumir de una tabla o de un índice). El rendimiento aprovisionado en cada índice es independiente del rendimiento aprovisionado en una tabla. Esto permite definir el rendimiento con detalle a fin de satisfacer las necesidades de la aplicación.

In [11]:
dt.check_status_gsi(index_name='DownloadsByMonth')

'ACTIVE'